In [1]:
from datetime import datetime
from lassort import load, localAssortF
import numpy as np

In [2]:
networkfile = '../../problem_sets/2/facebook100txt/Reed98.txt'
metadatafile = '../../problem_sets/2/facebook100txt/Reed98_attr.txt'


Ms = []
Ts = []
Zs = []

for col in range(1,6):
    start = datetime.now()
    E, M = load(
        networkfile, 
        metadatafile, 
        zero_index=1, 
        meta_col=col,
        header=True
    )
    assortM, assortT, Z = localAssortF(E,M,pr=np.arange(0,1,0.1))
    Ms.append(assortM)
    Ts.append(assortT)
    Zs.append(Z)
    end = datetime.now()
    print(f"Ran multiscale mixing for attribute {col} in {(end-start).total_seconds()} seconds.")

Ran multiscale mixing for attribute 1 in 6.317854 seconds.
Ran multiscale mixing for attribute 2 in 6.282173 seconds.


ValueError: column index exceeds matrix dimensions